In [2]:
!pip install fetoflow

In [3]:
from FetoFlow import *
import pandas as pd

ModuleNotFoundError: No module named 'fetoflow'

In [ ]:
# read in node and element files
node_filename= 'sample_geometry/full_tree.ipnode'
element_filename = 'sample_geometry/full_tree.ipelem'
radius_filename = 'sample_geometry/chorionic_element_radii_cycle3_v5_two_inlets.ipfiel'

In [ ]:
# process nodes and elements using functions from file_parsing_utils()
nodes = read_nodes(node_filename)
elements = read_elements(element_filename)
fields = define_fields_from_files(files={"radius":radius_filename})

In [ ]:
# define boundary conditions
inlet_pressure, outlet_pressure = 6650, 2660
# generate boundaray conditions dictionary 
print("Creating Boundary Conditions")
bcs = generate_boundary_conditions(inlet_pressure = inlet_pressure, outlet_pressure = outlet_pressure, inlet_flow=None)

In [ ]:
# define other required geometric features (radii and decay factors)
umbilical_artery_radius, decay_factor = 1.8 / 1000, 1.38 
umbilical_vein_radius, decay_factor_vein = 4.0 / 1000, 1.46
arteries_only = False # this should rarely be true

viscosity_type = 'constant' # can also be 'pries_network' or 'pries_vessel' if wanting to incorporate radius-dependence

In [ ]:
# Generate the di-graph & calculate the resistances based on the viscosity
print("Creating Geometry")
G = create_geometry(nodes, elements, umbilical_artery_radius, decay_factor, arteries_only, umbilical_vein_radius, decay_factor_vein,fields=fields)

In [ ]:
 print("Calculating Resistance")
G = calculate_resistance(G, viscosity_model=viscosity_type)

In [ ]:
# solve
print("Calculating Matrices")
A,b,bc_export = create_small_matrices(G,bcs,branching_angles=False)
print("Solving for Pressures and Flows")
p,q = solve_small_system(A,b,G,bc_export)

In [ ]:
# store pressure and flows
pressures = pd.DataFrame([{"Node" : node, "Pressure" : pressure} for node,pressure in p.items()])
pressures.to_csv("output_data/example_simulation_pressures.csv")
flows = pd.DataFrame([{"Element" : element, "Flow" : flow} for element,flow in q.items()])
flows.to_csv("output_data/example_simulation_flows.csv")

In [ ]:
pressures.head(10)

In [ ]:
flows.head(10)

In [ ]:
node_filename = "sample_geometry/FullTree.ipnode"
    element_filename = "sample_geometry/FullTree.ipelem"
    boundary_conditions = {
        "inlet_pressure" : 6650,
        "outlet_pressure" : 2660
    }
    inlet_radius = 1.8/1000
    strahler_ratio_arteries = 1.38

In [ ]:
# call pressure and flows function
G = pressures_and_flows(
    node_filename,
    element_filename,
    boundary_conditions,
    inlet_radius,
    strahler_ratio_arteries,
    input_directory=".",
    output_directory="./output_data",
    flow_output_filename="flow_values.csv",
    pressure_output_filename="pressure_values.csv",
    arteries_only=False,
    viscosity_model="constant",
    vessel_type="rigid",
    outlet_vein_radius=4.0,
    strahler_ratio_veins=1.46,
    anastomosis=None,
    mu=0.33600e-2,  # This is the non-capillary viscosity value used
    capillary_model="analytical2015",
    capillary_parameters=None, 
    radius_filename=None,
    other_field_filenames=None,  
    verbose=False,
    time_statistics=False,
    return_graph=True,
)
print(G)